# PE ARI Task Post-Processing

## Libraries

In [10]:
import os
import pickle
import json
import pandas as pd
import ast
import sys

In [2]:
#am_work/coling_2025/cdcp/finetuned_models_run3/CDCP_acc_mistral-7b-instruct-v0.3-bnb-4bit

In [25]:
# nb_epochs = 0.2
OUTPUT_DIR = "../finetuned_models//PE_acc_essay_wtags_Qwen2-7B-Instruct-bnb-4bit"

In [26]:
with open(os.path.join(OUTPUT_DIR, f"""PE_acc_results_5.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [27]:
results

{'ground_truths': ['{"component_types": ["MajorClaim", "Premise", "Premise", "Premise", "Claim", "Premise", "Premise", "Premise", "Claim", "Premise", "Premise", "Claim"]}',
  '{"component_types": ["MajorClaim", "Premise", "Premise", "Premise", "Claim", "Premise", "Premise", "Premise", "Premise", "Premise", "Claim", "MajorClaim", "Claim"]}',
  '{"component_types": ["MajorClaim", "MajorClaim", "Premise", "Premise", "Premise", "Premise", "Claim", "Premise", "Premise", "Premise", "Premise", "Premise", "Claim", "Premise", "Premise", "MajorClaim"]}',
  '{"component_types": ["MajorClaim", "Premise", "Premise", "Claim", "Premise", "Premise", "Premise", "Claim", "MajorClaim", "Claim"]}',
  '{"component_types": ["MajorClaim", "Premise", "Claim", "Premise", "Premise", "Claim", "Premise", "Premise", "Premise", "Premise", "Premise", "Premise", "Premise", "Premise", "Claim", "Claim", "Premise", "Premise", "Premise", "Premise", "MajorClaim", "Claim"]}',
  '{"component_types": ["MajorClaim", "Premise"

In [23]:
def get_ac_count(x):

    return len(ast.literal_eval(x.AC_types))

def get_all_possible_pairs(x):

    pairs_l = []

    ac_count = x.AC_count
    ar_pairs = ast.literal_eval(x.AR_pairs)
    
    for i in range(ac_count):
        for j in range(ac_count):
            if i != j:
                pair = (i, j)
                if pair in ar_pairs:
                    pairs_l.append([i, j, "Rel"])
                else:
                    pairs_l.append([i, j, "N-Rel"])

    return pairs_l

def process_preds(test_preds, LI_grounds_l):

    if len(test_preds) != len(LI_grounds_l):
        print("error in preds length, equalizing lengths ... ")

    preds_pairs_l = []

    for l_1, l_2 in zip(test_preds, LI_grounds_l):
        para_list = []
        for i, j, _ in l_2:
            pair = [i, j]
            if pair in l_1:
                para_list.append([i, j, "Rel"])
            else:
                para_list.append([i, j, "N-Rel"])

        preds_pairs_l.append(para_list)

    return preds_pairs_l

def post_process_ari(results):

    script_directory = os.getcwd()
    parent_directory = os.path.abspath(os.path.join(script_directory, os.pardir))
    data_directory = os.path.join(parent_directory, "data")
    print(parent_directory)
    pe_df = pd.read_csv(os.path.join(data_directory, "PE_data.csv"))
    df_split = pd.read_csv(os.path.join(data_directory, "train-test-split.csv"), sep=";") 
    pe_df['split'] = pe_df['essay_id'].map(df_split['SET'])
    pe_df["AC_count"] = pe_df.apply(lambda x: get_ac_count(x), axis=1)
    pe_df["LI_grounds"] = pe_df.apply(lambda x: get_all_possible_pairs(x), axis=1)

    preds = results["predictions"]
    preds = [x["content"] for x in preds]
    preds = [json.loads(x)["list_argument_relations"] for x in preds]    


    LI_grounds_l = list(pe_df[pe_df.split == "TEST"].reset_index().LI_grounds)
    LI_preds_l = process_preds(preds, LI_grounds_l)

    LI_grounds = [item for row in LI_grounds_l for item in row]
    LI_preds = [item for row in LI_preds_l for item in row]
        
    task_grounds = [x[2] for x in LI_grounds]
    task_preds = [x[2] for x in LI_preds]    

    return task_grounds, task_preds   

In [24]:
task_grounds, task_preds = post_process_ari(results)

/Utilisateurs/umushtaq/am_work/AMwithLLMs/pe


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
len(task_grounds), len(task_preds)

(4108, 4108)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(task_grounds, task_preds, digits=3))

              precision    recall  f1-score   support

       N-Rel      0.976     0.976     0.976      3384
         Rel      0.888     0.888     0.888       724

    accuracy                          0.961      4108
   macro avg      0.932     0.932     0.932      4108
weighted avg      0.961     0.961     0.961      4108



In [19]:
with open(f"""{OUTPUT_DIR}/classification_report.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(task_grounds, task_preds, output_dict=True), fh)

In [5]:
grounds_global = results["ground_truths"]

In [6]:
preds_global = results["predictions"]

In [7]:
len(grounds_global), len(preds_global)

(150, 150)

## ACC post-processing

In [8]:
grounds = [json.loads(x)["component_types"] for x in grounds_global]

In [9]:
preds = [x["content"] for x in preds_global]

In [10]:
#preds[75] = '{"component_types": ["value", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy"]}'

In [11]:
#preds[8] = '{"component_types": ["fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact"], "argument_relations_and_types": [[1, 2, "reason"], [1, 3, "reason"], [1, 5, "reason"], [1, 6, "reason"], [1, 7, "reason"], [1, 8, "reason"], [1, 9, "reason"], [1, 10, "reason"], [2, 3, "reason"], [2, 5, "reason"], [2, 6, "reason"], [2, 7, "reason"], [2, 8, "reason"], [2, 9, "reason"], [2, 10, "reason"], [3, 5, "reason"], [3, 6, "reason"], [3, 7, "reason"], [3, 8, "reason"], [3, 9, "reason"], [3, 10, "reason"], [4, 5, "reason"], [4, 6, "reason"], [4, 7, "reason"], [4, 8, "reason"], [4, 9, "reason"], [4, 10, "reason"], [5, 6, "reason"], [5, 7, "reason"], [5, 8, "reason"], [5, 9, "reason"], [5, 10, "reason"], [6, 7, "reason"], [6, 8, "reason"], [6, 9, "reason"], [6, 10, "reason"], [7, 8, "reason"], [7, 9, "reason"], [7, 10, "reason"], [8, 9, "reason"], [8, 10, "reason"], [9, 10, "reason"], [1, 4, "reason"], [2, 4, "reason"], [3, 4, "reason"], [5, 4, "reason"], [6, 4, "reason"], [7, 4, "reason"], [8, 4, "reason"], [9, 4, "reason"], [10, 4, "reason"], [1, 3, "reason"], [2, 3, "reason"], [3, 3, "reason"], [4, 3, "reason"], [5, 3, "reason"], [6, 3, "reason"], [7, 3, "reason"], [8, 3, "reason"], [9, 3, "reason"], [10, 3, "reason"], [1, 2, "reason"], [1, 5, "reason"], [1, 6, "reason"], [1, 7, "reason"], [1, 8, "reason"], [1, 9, "reason"], [1, 10, "reason"], [2, 5, "reason"], [2, 6, "reason"], [2, 7, "reason"], [2, 8, "reason"], [2, 9, "reason"], [2, 10, "reason"], [3, 5, "reason"], [3, 6, "reason"], [3, 7, "reason"], [3, 8, "reason"], [3, 9, "reason"], [3, 10, "reason"], [4, 5, "reason"], [4, 6, "reason"], [4, 7, "reason"], [4, 8, "reason"], [4, 9, "reason"], [4, 10, "reason"], [5, 6, "reason"], [5, 7, "reason"], [5, 8, "reason"], [5, 9, "reason"], [5, 10, "reason"], [6, 7, "reason"], [6, 8, "reason"], [6, 9, "reason"], [6, 10, "reason"], [7, 8, "reason"], [7, 9, "reason"], [7, 10, "reason"], [8, 9, "reason"], [8, 10, "reason"], [9, 10, "reason"], [1, 1, "reason"], [2, 2, "reason"], [3, 3, "reason"], [4, 4, "reason"], [5, 5, "reason"], [6, 6, "reason"], [7, 7, "reason"], [8, 8, "reason"]]}'

In [12]:
preds = [json.loads(x)["component_types"] for x in preds]

In [13]:
len(grounds), len(preds)

(150, 150)

In [14]:
grounds

[['fact', 'value', 'policy'],
 ['value', 'policy', 'policy', 'policy', 'policy'],
 ['policy', 'policy'],
 ['value', 'value'],
 ['value', 'value', 'value', 'testimony', 'value'],
 ['testimony', 'testimony', 'testimony', 'value', 'value', 'policy'],
 ['value', 'fact', 'testimony', 'testimony', 'testimony'],
 ['value', 'testimony', 'value', 'value', 'policy', 'policy'],
 ['testimony',
  'fact',
  'fact',
  'policy',
  'fact',
  'value',
  'fact',
  'fact',
  'fact',
  'value'],
 ['value', 'value', 'value', 'value'],
 ['fact',
  'value',
  'testimony',
  'testimony',
  'testimony',
  'policy',
  'value',
  'value'],
 ['fact', 'value', 'policy', 'value', 'policy', 'policy'],
 ['value', 'value', 'fact', 'value', 'testimony', 'policy'],
 ['testimony', 'fact', 'value', 'value'],
 ['value', 'value', 'value', 'value'],
 ['value', 'value', 'value', 'value'],
 ['policy', 'fact', 'fact', 'fact'],
 ['value', 'testimony', 'testimony', 'testimony'],
 ['testimony', 'testimony', 'testimony', 'testimony'

In [15]:
preds

[['fact', 'fact', 'policy'],
 ['fact', 'fact', 'fact', 'policy', 'value'],
 ['policy', 'policy'],
 ['value', 'value'],
 ['value', 'policy', 'fact', 'testimony', 'value'],
 ['testimony', 'testimony', 'testimony', 'value', 'policy', 'policy'],
 ['value', 'fact', 'testimony', 'testimony', 'testimony'],
 ['value', 'value', 'policy', 'value', 'policy', 'policy'],
 ['testimony',
  'value',
  'value',
  'policy',
  'fact',
  'value',
  'fact',
  'fact',
  'fact',
  'value'],
 ['value', 'fact', 'value', 'value'],
 ['fact',
  'value',
  'testimony',
  'testimony',
  'testimony',
  'policy',
  'value',
  'value'],
 ['fact', 'fact', 'policy', 'value', 'policy', 'policy'],
 ['value', 'value', 'fact', 'value', 'testimony', 'policy'],
 ['testimony', 'testimony', 'value', 'value'],
 ['value', 'policy', 'value', 'value'],
 ['value', 'value', 'value', 'value'],
 ['policy', 'testimony', 'testimony', 'testimony'],
 ['value', 'testimony', 'testimony', 'testimony'],
 ['testimony', 'testimony', 'value', 'va

In [16]:
def opposite(component_type):

    if component_type == "fact":
        return "value"
    elif component_type == "value":
        return "policy"
    elif component_type == "policy":
        return "value"
    elif component_type == "testimony":
        return "fact"
    elif component_type == "reference":
        return "policy"

In [17]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [18]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [19]:
ATC_preds = [item for row in preds for item in row]
ATC_grounds = [item for row in grounds for item in row]

In [20]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

In [21]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

        fact      0.575     0.583     0.579       132
      policy      0.894     0.830     0.861       153
   reference      1.000     1.000     1.000         1
   testimony      0.904     0.852     0.878       244
       value      0.823     0.861     0.841       496

    accuracy                          0.819      1026
   macro avg      0.839     0.825     0.832      1026
weighted avg      0.821     0.819     0.819      1026



In [23]:
with open(f"""{OUTPUT_DIR}/classification_report_acc.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(ATC_grounds, ATC_preds, output_dict=True), fh)

### ARI post-processing (TBD)

In [24]:
len(grounds_global), len(preds_global)

(150, 150)

In [25]:
def process_grounds_ari(grounds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in grounds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l


def process_preds_ari(preds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in preds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [26]:
import datasets
from datasets import load_dataset

In [27]:
cdcp_dataset = load_dataset("DFKI-SLT/cdcp", trust_remote_code=True)

nr_acs_l = []

for sample in cdcp_dataset['test']:
    nr_acs_l.append(len(sample['propositions']['label']))

In [28]:
grounds = [json.loads(x)["argument_relations_and_types"] for x in grounds_global]

preds = [x["content"] for x in preds_global]

preds = [json.loads(x)["argument_relations_and_types"] for x in preds]

In [29]:
grounds

[[[3, 2, 'reason']],
 [],
 [],
 [],
 [[5, 3, 'reason'], [5, 4, 'reason']],
 [[5, 1, 'reason'], [5, 2, 'reason'], [5, 3, 'reason'], [5, 4, 'reason']],
 [[2, 3, 'evidence'],
  [2, 4, 'evidence'],
  [2, 5, 'evidence'],
  [1, 4, 'reason'],
  [1, 5, 'reason']],
 [[1, 2, 'reason'], [4, 3, 'reason'], [6, 3, 'reason']],
 [[4, 5, 'reason'],
  [4, 6, 'reason'],
  [6, 8, 'reason'],
  [6, 9, 'reason'],
  [6, 10, 'reason']],
 [[1, 2, 'reason'], [1, 4, 'reason']],
 [[2, 3, 'reason'],
  [2, 4, 'reason'],
  [2, 5, 'reason'],
  [6, 7, 'reason'],
  [6, 8, 'reason']],
 [[3, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [5, 2, 'reason']],
 [],
 [[2, 3, 'reason'], [2, 4, 'reason']],
 [[1, 2, 'reason'], [3, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [],
 [],
 [[1, 3, 'reason'], [3, 2, 'reason'], [5, 6, 'reason']],
 [[2, 3, 'reason'], [2, 4, 'reason'], [7, 8, 'reason'], [8, 9, 'reason']],
 [[3, 4, 'reason']],
 [],
 

In [30]:
preds

[[[3, 1, 'reason']],
 [],
 [],
 [],
 [[2, 5, 'reason'], [3, 4, 'reason']],
 [],
 [],
 [],
 [[4, 6, 'reason'], [6, 7, 'reason'], [6, 8, 'reason'], [6, 9, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [],
 [[3, 4, 'reason'], [5, 3, 'reason'], [5, 6, 'reason']],
 [[1, 2, 'reason'], [6, 2, 'reason'], [6, 4, 'reason'], [6, 5, 'reason']],
 [],
 [[2, 3, 'reason'], [2, 4, 'reason']],
 [[1, 2, 'reason'], [3, 4, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [1, 4, 'reason']],
 [],
 [[3, 4, 'reason']],
 [[1, 2, 'reason']],
 [[1, 2, 'reason'], [1, 3, 'reason'], [3, 4, 'reason']],
 [[5, 8, 'reason'], [5, 9, 'reason'], [7, 8, 'reason'], [7, 9, 'reason']],
 [[1, 2, 'reason'], [4, 3, 'reason'], [6, 5, 'reason']],
 [],
 [],
 [[2, 3, 'reason'], [26, 25, 'reason']],
 [[2, 1, 'reason'], [3, 4, 'reason'], [6, 2, 'reason'], [6, 3, 'reason']],
 [],
 [],
 [],
 [[3, 6, 'reason'], [4, 6, 'reason']],
 [],
 [],
 [],
 [],
 [],
 [[2, 1, 'reason']],
 [[4, 5, 'reason'], [6, 4, 'reason']],


In [31]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            l_tmp.append(l[0:2])
            
        grounds_l.append(l_tmp)

    return grounds_l

In [32]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [33]:
len(grounds), len(preds)

(150, 150)

In [34]:
grounds

[[[3, 2]],
 [],
 [],
 [],
 [[5, 3], [5, 4]],
 [[5, 1], [5, 2], [5, 3], [5, 4]],
 [[2, 3], [2, 4], [2, 5], [1, 4], [1, 5]],
 [[1, 2], [4, 3], [6, 3]],
 [[4, 5], [4, 6], [6, 8], [6, 9], [6, 10]],
 [[1, 2], [1, 4]],
 [[2, 3], [2, 4], [2, 5], [6, 7], [6, 8]],
 [[3, 4]],
 [[1, 2], [1, 3], [5, 2]],
 [],
 [[2, 3], [2, 4]],
 [[1, 2], [3, 4]],
 [[1, 2], [1, 3], [1, 4]],
 [[1, 2], [1, 3], [1, 4]],
 [],
 [],
 [[1, 3], [3, 2], [5, 6]],
 [[2, 3], [2, 4], [7, 8], [8, 9]],
 [[3, 4]],
 [],
 [],
 [[2, 3],
  [2, 4],
  [2, 5],
  [9, 11],
  [9, 12],
  [9, 10],
  [14, 13],
  [26, 25],
  [26, 27]],
 [[5, 4]],
 [],
 [[2, 3], [2, 4], [5, 1], [5, 2], [6, 7], [7, 1], [7, 2]],
 [],
 [[1, 2], [5, 6]],
 [],
 [],
 [],
 [],
 [],
 [[2, 1]],
 [[1, 2], [1, 3], [4, 5], [6, 5]],
 [[1, 3], [1, 4], [1, 5], [6, 3], [6, 4], [6, 5]],
 [],
 [],
 [],
 [[1, 2]],
 [[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8]],
 [[2, 3], [2, 4], [2, 5]],
 [],
 [],
 [[4, 3], [7, 6], [8, 2], [8, 4]],
 [],
 [],
 [[1, 2], [1, 3], [5, 6]],
 

In [35]:
preds

[[[3, 1]],
 [],
 [],
 [],
 [[2, 5], [3, 4]],
 [],
 [],
 [],
 [[4, 6], [6, 7], [6, 8], [6, 9]],
 [[1, 2], [1, 3], [1, 4]],
 [],
 [[3, 4], [5, 3], [5, 6]],
 [[1, 2], [6, 2], [6, 4], [6, 5]],
 [],
 [[2, 3], [2, 4]],
 [[1, 2], [3, 4]],
 [[1, 2], [1, 3], [1, 4]],
 [],
 [[3, 4]],
 [[1, 2]],
 [[1, 2], [1, 3], [3, 4]],
 [[5, 8], [5, 9], [7, 8], [7, 9]],
 [[1, 2], [4, 3], [6, 5]],
 [],
 [],
 [[2, 3], [26, 25]],
 [[2, 1], [3, 4], [6, 2], [6, 3]],
 [],
 [],
 [],
 [[3, 6], [4, 6]],
 [],
 [],
 [],
 [],
 [],
 [[2, 1]],
 [[4, 5], [6, 4]],
 [[2, 3], [6, 3], [6, 4], [6, 5]],
 [[1, 2]],
 [],
 [],
 [[1, 2]],
 [],
 [[2, 3], [2, 4], [2, 5], [6, 7]],
 [],
 [[1, 2], [1, 3], [1, 4]],
 [[7, 5], [7, 6]],
 [],
 [],
 [[1, 3], [1, 5], [5, 2], [6, 3], [6, 5]],
 [[1, 2],
  [3, 2],
  [6, 5],
  [7, 8],
  [7, 9],
  [9, 10],
  [11, 10],
  [11, 12],
  [11, 13],
  [12, 13],
  [14, 12],
  [14, 13]],
 [[7, 8], [11, 10]],
 [[1, 2], [1, 3], [1, 4]],
 [[1, 2], [1, 3], [2, 3], [2, 4], [2, 5], [5, 4], [7, 6]],
 [[12, 11], [13, 1

In [36]:
final_grounds = process_grounds_ari(grounds, nr_acs_l)
final_preds = process_preds_ari(preds, nr_acs_l)

In [37]:
task_grounds = [x[2] for x in final_grounds]
task_preds = [x[2] for x in final_preds]

In [38]:
# sanity check: 
len(task_grounds) == len(task_preds)

True

In [39]:
print(classification_report(task_grounds, task_preds, digits=3))

              precision    recall  f1-score   support

       N-Rel      0.985     0.988     0.986     10078
         Rel      0.434     0.380     0.405       250

    accuracy                          0.973     10328
   macro avg      0.709     0.684     0.696     10328
weighted avg      0.971     0.973     0.972     10328



In [40]:
with open(f"""{OUTPUT_DIR}/classification_report_ari.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(task_grounds, task_preds, output_dict=True), fh)

## ARC post-processing

In [41]:
grounds = [json.loads(x)["argument_relations_and_types"] for x in grounds_global]
preds = [x["content"] for x in preds_global]

In [42]:
preds = [json.loads(x)["argument_relations_and_types"] for x in preds]

In [43]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            try:
                l_tmp.append(l[2])
            except:
                l_tmp.append("Support")
            
        grounds_l.append(l_tmp)

    return grounds_l

In [44]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [45]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    if len(x) != len(y):
        print(i,x,y)

5 ['reason', 'reason', 'reason', 'reason'] []
6 ['evidence', 'evidence', 'evidence', 'reason', 'reason'] []
7 ['reason', 'reason', 'reason'] []
8 ['reason', 'reason', 'reason', 'reason', 'reason'] ['reason', 'reason', 'reason', 'reason']
9 ['reason', 'reason'] ['reason', 'reason', 'reason']
10 ['reason', 'reason', 'reason', 'reason', 'reason'] []
11 ['reason'] ['reason', 'reason', 'reason']
12 ['reason', 'reason', 'reason'] ['reason', 'reason', 'reason', 'reason']
17 ['reason', 'reason', 'reason'] []
18 [] ['reason']
19 [] ['reason']
22 ['reason'] ['reason', 'reason', 'reason']
25 ['reason', 'reason', 'reason', 'reason', 'reason', 'reason', 'reason', 'reason', 'reason'] ['reason', 'reason']
26 ['reason'] ['reason', 'reason', 'reason', 'reason']
28 ['evidence', 'evidence', 'reason', 'reason', 'reason', 'reason', 'reason'] []
37 ['reason', 'reason', 'reason', 'reason'] ['reason', 'reason']
38 ['reason', 'reason', 'reason', 'reason', 'reason', 'reason'] ['reason', 'reason', 'reason', 'rea

In [46]:
def opposite(relation_type):
    
    if relation_type == "reason":
        return "evidence"
    else:
        return "reason"

In [47]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds  

In [48]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [49]:
LTC_preds = [item for row in preds for item in row]
LTC_grounds = [item for row in grounds for item in row]

In [50]:
# sanity check: 
len(LTC_preds) == len(LTC_grounds)

True

In [51]:
print(classification_report(LTC_grounds, LTC_preds, digits=3))

              precision    recall  f1-score   support

    evidence      0.056     0.269     0.093        26
      reason      0.905     0.607     0.727       298

    accuracy                          0.580       324
   macro avg      0.481     0.438     0.410       324
weighted avg      0.837     0.580     0.676       324



In [52]:
with open(f"""{OUTPUT_DIR}/classification_report_arc.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(LTC_grounds, LTC_preds, output_dict=True), fh)